In [14]:
import os
import sys
import asyncio
from async_timeout import timeout
import json

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from src.clients.morpho_client import MorphoClient


async def run_with_timeout(coro, timeout_seconds=10):
    async with timeout(timeout_seconds):
        return await coro


In [15]:
# Setup client
RPC = "ETHEREUM"  # ARBITRUM or ETHEREUM
RPC_URL = os.getenv(f"{RPC}_RPC_URL")
user_address = os.getenv('WALLET_ADDRESS')

# Initialize client
morpho_client = MorphoClient(wallet_address=user_address,
                             private_key=os.getenv('PRIVATE_WALLET_KEY'),
                             network=RPC,
                             rpc_url=RPC_URL)

In [16]:
# Test get the smart contracts:
morpho_blue = morpho_client._get_morpho_contract()
print(morpho_blue.address)

morpho_irm = morpho_client._get_irm_contract()
print(morpho_irm.address)

0xBBBBBbbBBb9cC5e90e3b3Af64bdAF62C37EEFFCb
0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC


### Fetch a single market information

In [20]:
market_key = 'wstETH-USDT1' # see utils.morpho_markets for all available markets
market_id = morpho_client.get_market_info(market_key)
print(market_id)

0xe7e9694b754c4d4f7e21faf7223f6fa71abaeb10296a4c43a54a7977149687d2


In [21]:
markets_data = await run_with_timeout(morpho_client.fetch_market_data(market_id, user_address))
markets_data

market_params: MarketParams(loan_token='0xdAC17F958D2ee523a2206206994597C13D831ec7', collateral_token='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', oracle='0x95DB30fAb9A3754e42423000DF27732CB2396992', irm='0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC', lltv=860000000000000000)
position_user: UserPosition(supply_shares=0, borrow_shares=0, collateral=0)


{'supply_apy': 0.016405800664243676,
 'borrow_apy': 0.02678261686958068,
 'borrow_assets_user': 0,
 'market_total_supply': 3656123802523,
 'market_total_borrow': 2239573474096,
 'health_factor': 1.1579208923731619e+41,
 'is_healthy': True}

### Fetch many markets information

In [19]:
# List of market keys you want to fetch data for
market_keys = ['wstETH-USDT1', 'wstETH-USDT2', 'wstETH-USDC', 'wstETH-DAI', 'wstETH-USDA']

# Get market IDs for all markets
market_ids = [morpho_client.get_market_info(key) for key in market_keys]

# Fetch data for all markets
markets_data = await run_with_timeout(morpho_client.fetch_all_markets_data(market_ids, user_address))

# Print or process the results
for key, data in zip(market_keys, markets_data.values()):
    print(f"Market: {key}")
    print(json.dumps(data, indent=2))
    print("\n")

market_params: MarketParams(loan_token='0xdAC17F958D2ee523a2206206994597C13D831ec7', collateral_token='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', oracle='0x95DB30fAb9A3754e42423000DF27732CB2396992', irm='0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC', lltv=860000000000000000)
position_user: UserPosition(supply_shares=0, borrow_shares=0, collateral=0)
market_params: MarketParams(loan_token='0xdAC17F958D2ee523a2206206994597C13D831ec7', collateral_token='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', oracle='0xA1bDbfD967620332b9Fd7D60885e19EBBE49B882', irm='0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC', lltv=860000000000000000)
position_user: UserPosition(supply_shares=0, borrow_shares=0, collateral=0)
market_params: MarketParams(loan_token='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', collateral_token='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', oracle='0x48F7E36EB6B826B2dF4B2E630B62Cd25e89E40e2', irm='0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC', lltv=860000000000000000)
position_user: UserP